## NYPD Data

### Imports

In [ ]:
# data import
import pickle

# data manipulation
import numpy as np
import pandas as pd
from datetime import datetime

# plotting
import matplotlib.pyplot as plt

#gaussian filter
from scipy.ndimage import gaussian_filter

### Data Import

In [ ]:
pickle_path = ('../raw_data/clean.pickle')
with open(pickle_path, 'rb') as f:
    df = pickle.load(f)
del pickle_path, f

### Sample Selection

In [ ]:
inf = df['period'] > datetime(2016, 10, 1, 0, 0, 0)
sup = df['period'] < datetime(2016, 11, 1, 0, 0, 0)
sample = df[ inf & sup ]
del inf, sup, df

### Lat - Long to Array

In [ ]:
def from_meters_to_coords(lat_meters, lon_meters):
    """
    gives the latitude and longitude step to use for the grid buckets
    lat_meters, lon_meters = lat/lon step
    """
    #Position, decimal degrees
    lat = 40
    lon = -73

    #Earth’s radius, sphere
    R=6378137

    #offsets in meters
    dn = lat_meters
    de = lon_meters

    #Coordinate offsets in radians
    dLat = dn/R
    dLon = de/(R*np.cos(np.pi*lat/180))

    #OffsetPosition, decimal degrees
    latO = dLat * 180/np.pi
    lonO = dLon * 180/np.pi 

    del lat, lon, R, dn, de, dLat, dLon

    return latO, lonO

In [ ]:
def from_coord_to_matrix(df, lat_meters, lon_meters):
    """
    outputs the 3D matrix of all coordinates for a given bucket height and width in meters
    """
    df=df.copy()
    #add 'time_index' column to df
    ind = {time:index for index,time in enumerate(np.sort(df['period'].unique()))}
    df['time_index'] = df['period'].map(ind)
    
    #initiate matrix
    grid_offset = np.array([0, -40.91553277600008,  -74.25559136315213,]) # Where do you start
    #from meters to lat/lon step
    lat_spacing, lon_spacing = from_meters_to_coords(lat_meters, lon_meters )
    grid_spacing = np.array([1, lat_spacing , lon_spacing]) # What's the space you consider (euclidian here)
     
    
    #get points coordinates
    coords = np.array([(t_ind, -lat, lon) for t_ind, lat, lon \
                   in zip(df['time_index'],df['latitude'],df['longitude'])])
    
    
    # Convert point to index
    indexes = np.round((coords - grid_offset)/grid_spacing).astype('int')
    Z = indexes[:,0]
    Y = indexes[:,1]
    X = indexes[:,2]
    
    #virgin matrix
    a = np.zeros((Z.max()+1, Y.max()+1, X.max()+1))
   
    
    a[Z, Y, X]=1
    
    del ind, grid_offset, lat_spacing, lon_spacing, grid_spacing, coords, indexes, Z, Y, X
    
    return a, a.shape[1], a.shape[2]

In [ ]:
img, lat_size, lon_size = from_coord_to_matrix(sample, 100, 100)
del sample

### Plotting Array (with and w/o Gaussian filter)

In [ ]:
#plt.figure(figsize=(5, 5))
#plt.imshow(img[3], cmap='gray');

In [ ]:
img3D_conv = gaussian_filter(img, sigma=(2, 2, 2))
del img

plt.figure(figsize=(5, 5))

plt.imshow(img3D_conv[3], cmap='gray');

### Splitting into X and y

In [ ]:
def get_observation_target(img3D_conv, x_length, y_length):
    '''
    output an observation of x_length consecutive images and the y_length next images as the target
    '''
    position = np.random.randint(0, img3D_conv.shape[0] - (x_length + y_length))
    observation = img3D_conv[position:position + x_length]
    target = img3D_conv[position + x_length:position + (x_length + y_length)]
    
    del position 

    return observation, target

### Subsampling to get multiple X and y

In [ ]:
def get_X_y(img3D_conv, number_of_observations, x_length, y_length):
    '''
    outputs n observations and their associated targets
    '''
    X = []
    y = []
    
    for n in range(number_of_observations):
        X_subsample, y_subsample = get_observation_target(img3D_conv, x_length, y_length)
        X.append(X_subsample)
        y.append(y_subsample)
        
    X = np.array(X)
    y = np.array(y)

    del X_subsample, y_subsample, n
    
    return X, y

In [ ]:
x_length = 24
y_length= 1

X, y = get_X_y(img3D_conv, 50, x_length, y_length)
del img3D_conv

In [ ]:
y.shape

In [ ]:
y[0]

## Model

### Imports

In [ ]:
# Train, test, split
from sklearn.model_selection import train_test_split

# Model
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

### Train, test, split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
del X,y

### Model

In [ ]:
def build_encoder(latent_dimension, x_length, lat_size, lon_size):
    encoder = models.Sequential()
    
    print('adding conv2D 1')
    encoder.add(layers.Conv2D(64, kernel_size = 5, activation = 'relu',padding='same',
                            input_shape = (x_length, lat_size, lon_size),
                           data_format='channels_first'))
    encoder.add(layers.MaxPooling2D(2, data_format='channels_first'))

    print('adding conv2D 2')
    encoder.add(layers.Conv2D(128, kernel_size = 3, activation = 'relu', padding='same',  data_format='channels_first'))
    encoder.add(layers.MaxPooling2D(2, data_format='channels_first'))

    print('adding conv2D 3')
    encoder.add(layers.Conv2D(64, kernel_size = 3, activation = 'relu', padding='same', data_format='channels_first' ))
    encoder.add(layers.MaxPooling2D(2, data_format='channels_first'))     

    print('flattening')
    encoder.add(layers.Flatten())
    
    print('adding dense layer 1')
    encoder.add(layers.Dense(20, activation = 'relu'))
    
    print('adding latent layer')
    encoder.add(layers.Dense(latent_dimension, activation='tanh'))
    
    return encoder

In [ ]:
encoder = build_encoder(2, )
encoder.summary()

In [ ]:
def build_decoder(latent_dimension):
    decoder = models.Sequential()
    
    decoder.add(layers.Dense(8*57*59, activation='tanh', input_shape=(latent_dimension,)))
    decoder.add(layers.Reshape((8, 57, 59)))  #no batch axis here

    decoder.add(layers.Conv2DTranspose(8, (2, 2), strides=2, padding='same', activation='relu',data_format='channels_first'))
    decoder.add(layers.Conv2DTranspose(8, (2, 2), strides=2, padding='same', activation='relu',data_format='channels_first'))
    
    decoder.add(layers.Conv2DTranspose(1, (2, 2), strides=2, padding='same', activation='relu',data_format='channels_first'))
    
    return decoder

In [ ]:
decoder = build_decoder(2)
decoder.summary()

In [ ]:
def model(x_length, y_length, lat_size, lon_size):
    
    print('initializing model')
    model = models.Sequential()
    
    #print('adding gaussian noise')
    #model.add(layers.GaussianNoise(stddev = 2.5, input_shape = (x_length, lat_size, lon_size)))
    
    print('adding conv2D 1')
    model.add(layers.Conv2D(64, kernel_size = 5, activation = 'relu',padding='same',
                            input_shape = (x_length, lat_size, lon_size),
                           data_format='channels_first'))
    model.add(layers.MaxPooling2D(2, data_format='channels_first'))
    
    #print('adding conv2D 2')
    #model.add(layers.Conv2D(128, kernel_size = 3, activation = 'relu', padding='same',  data_format='channels_first'))
    #model.add(layers.MaxPooling2D(2, data_format='channels_first'))
    
    #print('adding conv2D 3')
    #model.add(layers.Conv2D(256, kernel_size = 3, activation = 'relu', padding='same' ,data_format='channels_first'))
    #model.add(layers.MaxPooling2D(2, data_format='channels_first'))
    
    #print('adding conv2D 3')
    #model.add(layers.Conv2D(64, kernel_size = 3, activation = 'relu', padding='same', data_format='channels_first' ))
    #model.add(layers.MaxPooling2D(2, data_format='channels_first'))
    
    print('flattening')
    model.add(layers.Flatten())
    
    print('adding dense layer 1')
    model.add(layers.Dense(20, activation = 'relu'))

    #print('adding dense layer 2')
    #model.add(layers.Dense(, activation = 'relu'))

    print('adding latent layer')
    model.add(layers.Dense(1, activation = 'tanh'))
    
    print('adding Dense layer')
    model.add(layers.Dense(7*7*8, activation='tanh', input_shape=(1,)))
    
    print('adding déconv2D 1')
    model.add(layers.Conv2DTranspose(64, kernel_size = 3, activation = 'relu', padding='same', data_format='channels_first' ))

    
    print('compiling')
    model.compile(loss = 'mse',
                      optimizer = 'adam',
                      metrics = 'mae')
    print('Done !')
    return model

### Running Model (Instance, Early Stopping, Fit and Evaluate)

**Reminders**

**batch_size:** no. of data used to compute error between y_pred and y_true each time weights of neural network updated (use 16 or 32)

**epoch:** once all data has been used once to update the weights

In [ ]:
x_length = 24
y_length =1
lat_size = 460
lon_size = 473


In [ ]:
model = model(x_length,y_length, lat_size, lon_size)

In [ ]:
model.summary()

In [ ]:
es = EarlyStopping(patience = 1, restore_best_weights=True)

history = model.fit(X_train, y_train,
                      batch_size = 40, 
                      epochs = 100,
                      validation_split = 0.3,
                      callbacks = es)

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history['val_loss'])

In [ ]:
plt.plot(history.history['val_mae'])

In [ ]:
print(model.evaluate(X_test, y_test))

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Conv2D(50, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3), padding='same'))
model.add(layers.MaxPooling2D(2))
model.add(layers.Conv2D(30, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(50, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3), padding='same'))
model.add(layers.MaxPooling2D(2))
model.add(layers.Conv2D(20, kernel_size=3, activation='relu', padding='same'))   # Note that kernel_size=3 corresponds to (3, 3)
model.add(layers.Conv2D(10, kernel_size=3, activation='relu', padding='same'))
model.add(layers.Flatten())
model.add(layers.Dense(4, activation='softmax')) 

model.summary() # Compare the number of parameters to the model with Dense layers
